In [4]:
import pandas as pd
import tqdm
import warnings 
import glob
import numpy as np
from tqdm import tqdm
import seaborn as sns
import os
import tensorflow as tf
warnings.filterwarnings("ignore")

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM, CuDNNLSTM
from keras.optimizers import SGD
import keras.backend.tensorflow_backend as KTF

Using TensorFlow backend.


In [6]:
from skmultilearn.dataset import load_dataset
from skmultilearn.adapt import MLkNN

In [7]:
#进行配置，每个GPU使用60%上限现存
os.environ["CUDA_VISIBLE_DEVICES"]="0" # 使用编号为1，2号的GPU
config = tf.compat.v1.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 每个GPU现存上届控制在60%以内
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

# 设置session
#KTF.set_session(session)
#tf.keras.backend.set_session(sess)

## X Train

In [8]:
all_train_cases = os.listdir('D:\\DeepMind\\合肥高新杯心电人机智能大赛\\train\\')
all_train_cases.sort(key=lambda x:int(x[:-4]))
all_train_cases

['2.txt',
 '5.txt',
 '9.txt',
 '12.txt',
 '14.txt',
 '16.txt',
 '17.txt',
 '21.txt',
 '23.txt',
 '24.txt',
 '26.txt',
 '28.txt',
 '30.txt',
 '32.txt',
 '34.txt',
 '36.txt',
 '40.txt',
 '41.txt',
 '49.txt',
 '53.txt',
 '55.txt',
 '57.txt',
 '60.txt',
 '61.txt',
 '63.txt',
 '65.txt',
 '68.txt',
 '71.txt',
 '72.txt',
 '76.txt',
 '78.txt',
 '81.txt',
 '84.txt',
 '86.txt',
 '89.txt',
 '90.txt',
 '92.txt',
 '95.txt',
 '97.txt',
 '98.txt',
 '100.txt',
 '102.txt',
 '105.txt',
 '108.txt',
 '110.txt',
 '112.txt',
 '114.txt',
 '116.txt',
 '118.txt',
 '121.txt',
 '123.txt',
 '124.txt',
 '126.txt',
 '128.txt',
 '130.txt',
 '132.txt',
 '135.txt',
 '137.txt',
 '141.txt',
 '144.txt',
 '145.txt',
 '147.txt',
 '149.txt',
 '151.txt',
 '154.txt',
 '156.txt',
 '158.txt',
 '160.txt',
 '161.txt',
 '163.txt',
 '165.txt',
 '167.txt',
 '169.txt',
 '171.txt',
 '173.txt',
 '175.txt',
 '177.txt',
 '179.txt',
 '182.txt',
 '184.txt',
 '188.txt',
 '189.txt',
 '191.txt',
 '194.txt',
 '197.txt',
 '199.txt',
 '203.txt',

In [9]:
x_train = np.empty(shape=[24106, 5000, 12], dtype=np.int64)

In [10]:
for idx, sample in enumerate(tqdm(all_train_cases)):
    data = pd.read_table('train\\'+sample,header=0, encoding='utf-8', sep=' ')
    i = data['I']
    ii = data['II']
    data['III'] = ii-i
    data['aVR']=-(ii+i)/2
    data['aVL']=(i-ii)/2
    data['aVF']=(ii-i)/2
    x_train[idx] = data.values

100%|███████████████████████████████████████████████████████████████████████████| 24106/24106 [03:19<00:00, 120.98it/s]


In [11]:
x_train[0]

array([[-5,  1,  9, ...,  2, -3,  3],
       [-4,  1,  9, ...,  1, -2,  2],
       [-3,  1,  9, ...,  1, -2,  2],
       ...,
       [ 4,  2, 27, ..., -3,  1, -1],
       [ 6,  3, 28, ..., -4,  1, -1],
       [ 7,  4, 29, ..., -5,  1, -1]], dtype=int64)

## Y Train

In [12]:
label = pd.read_csv('hf_round1_label.txt',sep="\t",encoding='utf-8',names = list(range(0,12)))
label.columns = ['id','age','gender','f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10','f11']
arrythmia = pd.read_csv('hf_round1_arrythmia.txt',header=None,sep="\t",encoding='utf-8')
arrythmia.columns = ['ff1']
all_dis = list(arrythmia.ff1.unique())

In [13]:
dis_df = pd.DataFrame(np.zeros([label.shape[0],55]))
dis_df.columns = all_dis
new_data = pd.concat([label,dis_df],axis =1)
for i in tqdm(range (new_data.shape[0])):
    dises = list(label.iloc[i,3:11].unique())
    dises.remove(dises[-1])
    for d in dises:
        new_data.loc[i,d]=1

100%|███████████████████████████████████████████████████████████████████████████| 24106/24106 [00:26<00:00, 898.46it/s]


In [14]:
new_cols = [col for col in new_data.columns if col not in ['id','age','gender','f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10','f11']]#最后过滤掉不需要的列new_data= new_data[new_cols]
y_train = new_data[new_cols]
y_train = y_train.values

In [15]:
y_train.shape

(24106, 55)

## X Test

In [16]:
all_test_cases = os.listdir('D:\\DeepMind\\合肥高新杯心电人机智能大赛\\testA\\')
all_test_cases.sort(key=lambda x:int(x[:-4]))
all_test_cases

['1.txt',
 '3.txt',
 '6.txt',
 '7.txt',
 '10.txt',
 '13.txt',
 '15.txt',
 '20.txt',
 '22.txt',
 '35.txt',
 '54.txt',
 '58.txt',
 '69.txt',
 '74.txt',
 '75.txt',
 '77.txt',
 '79.txt',
 '83.txt',
 '115.txt',
 '125.txt',
 '127.txt',
 '129.txt',
 '131.txt',
 '134.txt',
 '136.txt',
 '140.txt',
 '142.txt',
 '146.txt',
 '148.txt',
 '150.txt',
 '155.txt',
 '162.txt',
 '164.txt',
 '166.txt',
 '168.txt',
 '172.txt',
 '174.txt',
 '178.txt',
 '181.txt',
 '187.txt',
 '190.txt',
 '193.txt',
 '198.txt',
 '215.txt',
 '216.txt',
 '221.txt',
 '232.txt',
 '235.txt',
 '244.txt',
 '251.txt',
 '255.txt',
 '259.txt',
 '277.txt',
 '285.txt',
 '287.txt',
 '294.txt',
 '296.txt',
 '298.txt',
 '301.txt',
 '304.txt',
 '306.txt',
 '308.txt',
 '312.txt',
 '319.txt',
 '321.txt',
 '325.txt',
 '330.txt',
 '336.txt',
 '343.txt',
 '348.txt',
 '350.txt',
 '353.txt',
 '355.txt',
 '357.txt',
 '359.txt',
 '366.txt',
 '370.txt',
 '390.txt',
 '392.txt',
 '396.txt',
 '399.txt',
 '408.txt',
 '412.txt',
 '414.txt',
 '422.txt',
 '

In [17]:
x_test = np.empty(shape=[8036, 5000, 12], dtype=np.int64)

In [18]:
for idx, sample in enumerate(tqdm(all_test_cases)):
    data = pd.read_table('testA\\'+sample,header=0, encoding='utf-8', sep=' ')
    i = data['I']
    ii = data['II']
    data['III'] = ii-i
    data['aVR']=-(ii+i)/2
    data['aVL']=(i-ii)/2
    data['aVF']=(ii-i)/2
    x_test[idx] = data.values

100%|█████████████████████████████████████████████████████████████████████████████| 8036/8036 [01:04<00:00, 124.56it/s]


In [19]:
x_test[0]

array([[  4,  17,  -6, ..., -10,  -6,   6],
       [  3,  15,  -6, ...,  -9,  -6,   6],
       [  3,  17,  -7, ..., -10,  -7,   7],
       ...,
       [ -5, -11,   0, ...,   8,   3,  -3],
       [ -5, -11,   0, ...,   8,   3,  -3],
       [ -5, -11,   0, ...,   8,   3,  -3]], dtype=int64)

## X&Y Dev

In [20]:
per = np.random.permutation(y_train.shape[0])
x_train = x_train[per,:,:]
y_train = y_train[per]

In [34]:
x_train_ = x_train[:20000]
y_train_ = y_train[:20000]
x_dev_ = x_train[20000:]
y_dev_ = y_train[20000:]

## LSTM Model

In [18]:
model = Sequential()
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(55, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.fit(x_train_, y_train_, batch_size=32, epochs=5)

Epoch 1/5
 4480/20000 [=====>........................] - ETA: 44:19 - loss: 0.7146 - accuracy: 0.535 - ETA: 40:10 - loss: 0.7118 - accuracy: 0.534 - ETA: 38:19 - loss: 0.7116 - accuracy: 0.533 - ETA: 37:48 - loss: 0.7073 - accuracy: 0.539 - ETA: 37:16 - loss: 0.7078 - accuracy: 0.537 - ETA: 37:06 - loss: 0.7069 - accuracy: 0.539 - ETA: 36:37 - loss: 0.7055 - accuracy: 0.540 - ETA: 36:18 - loss: 0.7042 - accuracy: 0.543 - ETA: 36:01 - loss: 0.7028 - accuracy: 0.545 - ETA: 35:47 - loss: 0.7004 - accuracy: 0.549 - ETA: 35:42 - loss: 0.6990 - accuracy: 0.549 - ETA: 35:38 - loss: 0.6968 - accuracy: 0.552 - ETA: 35:30 - loss: 0.6939 - accuracy: 0.556 - ETA: 35:21 - loss: 0.6929 - accuracy: 0.557 - ETA: 35:11 - loss: 0.6916 - accuracy: 0.558 - ETA: 35:11 - loss: 0.6902 - accuracy: 0.560 - ETA: 35:05 - loss: 0.6888 - accuracy: 0.561 - ETA: 34:57 - loss: 0.6872 - accuracy: 0.563 - ETA: 34:52 - loss: 0.6855 - accuracy: 0.566 - ETA: 34:44 - loss: 0.6838 - accuracy: 0.568 - ETA: 34:39 - loss: 0.68

KeyboardInterrupt: 

In [19]:
score = model.evaluate(x_dev_, y_dev_, batch_size=32)
score

4106/4106 [==============================] - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 58s - ETA: 57 - ETA: 55 - ETA: 54 - ETA: 52 - ETA: 51 - ETA: 49 - ETA: 48 - ETA:

[0.24333301540763566, 0.9633219242095947]

In [2]:
y_test = model.predict(x_test, batch_size=32)

NameError: name 'model' is not defined

In [23]:
y_test[20]

array([0.6766484 , 0.35022932, 0.16154328, 0.28941578, 0.28497666,
       0.22030514, 0.24055925, 0.20772591, 0.1482819 , 0.48427814,
       0.28301758, 0.22382769, 0.24948785, 0.25200352, 0.20208713,
       0.21583503, 0.21663427, 0.22100121, 0.24663994, 0.287032  ,
       0.15702209, 0.24644867, 0.26547974, 0.25943142, 0.3930442 ,
       0.24795714, 0.24410117, 0.19096336, 0.23484051, 0.23680454,
       0.20235676, 0.25933266, 0.22119838, 0.25082195, 0.15721476,
       0.38187242, 0.31222337, 0.24417636, 0.2646821 , 0.1180208 ,
       0.18628877, 0.18242243, 0.21291104, 0.22750565, 0.35375094,
       0.152069  , 0.12692443, 0.14713252, 0.14009166, 0.18553552,
       0.30090833, 0.31592965, 0.1395658 , 0.19307888, 0.11731905],
      dtype=float32)

## CNN Model

In [31]:
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalAveragePooling1D

In [78]:
model = Sequential()
model.add(Conv1D(200, 12, activation='relu', input_shape=(5000, 12)))
model.add(Conv1D(200, 12, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(100, 12, activation='relu'))
model.add(Conv1D(100, 12, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(55, activation='sigmoid'))


model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.fit(x_train_, y_train_, batch_size=32, epochs=5)

Epoch 1/5


 5856/20000 [=======>......................] - ETA: 5:28 - loss: 11.8277 - accuracy: 0.062 - ETA: 4:09 - loss: 11.2758 - accuracy: 0.296 - ETA: 3:39 - loss: 10.8591 - accuracy: 0.416 - ETA: 3:24 - loss: 11.2181 - accuracy: 0.484 - ETA: 3:15 - loss: 11.5114 - accuracy: 0.550 - ETA: 3:09 - loss: 11.0502 - accuracy: 0.588 - ETA: 3:04 - loss: 11.1830 - accuracy: 0.584 - ETA: 3:01 - loss: 11.0571 - accuracy: 0.609 - ETA: 2:58 - loss: 10.9991 - accuracy: 0.614 - ETA: 2:56 - loss: 10.9321 - accuracy: 0.615 - ETA: 2:54 - loss: 10.9780 - accuracy: 0.625 - ETA: 2:52 - loss: 10.8731 - accuracy: 0.630 - ETA: 2:51 - loss: 10.8460 - accuracy: 0.637 - ETA: 2:50 - loss: 10.7920 - accuracy: 0.627 - ETA: 2:49 - loss: 10.9810 - accuracy: 0.637 - ETA: 2:48 - loss: 11.1251 - accuracy: 0.642 - ETA: 2:47 - loss: 11.2350 - accuracy: 0.645 - ETA: 2:46 - loss: 11.1951 - accuracy: 0.649 - ETA: 2:45 - loss: 11.0695 - accuracy: 0.651 - ETA: 2:45 - loss: 11.0762 - accuracy: 0.653 - ETA: 2:44 - loss: 11.1994 - accur

11808/20000 [================>.............] - ETA: 1:57 - loss: 7.7923 - accuracy: 0.58 - ETA: 1:56 - loss: 7.7827 - accuracy: 0.58 - ETA: 1:56 - loss: 7.7829 - accuracy: 0.58 - ETA: 1:56 - loss: 7.7783 - accuracy: 0.58 - ETA: 1:56 - loss: 7.7712 - accuracy: 0.58 - ETA: 1:55 - loss: 7.7640 - accuracy: 0.58 - ETA: 1:55 - loss: 7.7566 - accuracy: 0.58 - ETA: 1:55 - loss: 7.7475 - accuracy: 0.58 - ETA: 1:55 - loss: 7.7416 - accuracy: 0.58 - ETA: 1:54 - loss: 7.7311 - accuracy: 0.59 - ETA: 1:54 - loss: 7.7234 - accuracy: 0.59 - ETA: 1:54 - loss: 7.7102 - accuracy: 0.59 - ETA: 1:53 - loss: 7.6991 - accuracy: 0.59 - ETA: 1:53 - loss: 7.6972 - accuracy: 0.59 - ETA: 1:53 - loss: 7.6941 - accuracy: 0.59 - ETA: 1:53 - loss: 7.6856 - accuracy: 0.59 - ETA: 1:52 - loss: 7.6818 - accuracy: 0.59 - ETA: 1:52 - loss: 7.6724 - accuracy: 0.59 - ETA: 1:52 - loss: 7.6673 - accuracy: 0.59 - ETA: 1:52 - loss: 7.6592 - accuracy: 0.59 - ETA: 1:51 - loss: 7.6495 - accuracy: 0.59 - ETA: 1:51 - loss: 7.6473 - ac

17760/20000 [=========================>....] - ETA: 1:09 - loss: 6.9757 - accuracy: 0.64 - ETA: 1:08 - loss: 6.9701 - accuracy: 0.64 - ETA: 1:08 - loss: 6.9684 - accuracy: 0.64 - ETA: 1:08 - loss: 6.9664 - accuracy: 0.64 - ETA: 1:08 - loss: 6.9640 - accuracy: 0.64 - ETA: 1:08 - loss: 6.9625 - accuracy: 0.64 - ETA: 1:07 - loss: 6.9611 - accuracy: 0.64 - ETA: 1:07 - loss: 6.9564 - accuracy: 0.64 - ETA: 1:07 - loss: 6.9525 - accuracy: 0.64 - ETA: 1:06 - loss: 6.9511 - accuracy: 0.64 - ETA: 1:06 - loss: 6.9485 - accuracy: 0.64 - ETA: 1:06 - loss: 6.9465 - accuracy: 0.64 - ETA: 1:06 - loss: 6.9425 - accuracy: 0.64 - ETA: 1:05 - loss: 6.9390 - accuracy: 0.64 - ETA: 1:05 - loss: 6.9345 - accuracy: 0.64 - ETA: 1:05 - loss: 6.9322 - accuracy: 0.64 - ETA: 1:04 - loss: 6.9304 - accuracy: 0.64 - ETA: 1:04 - loss: 6.9249 - accuracy: 0.64 - ETA: 1:04 - loss: 6.9215 - accuracy: 0.64 - ETA: 1:04 - loss: 6.9205 - accuracy: 0.64 - ETA: 1:03 - loss: 6.9195 - accuracy: 0.64 - ETA: 1:03 - loss: 6.9169 - ac

20000/20000 [==============================] - ETA: 18s - loss: 6.5988 - accuracy: 0.660 - ETA: 18s - loss: 6.5968 - accuracy: 0.660 - ETA: 18s - loss: 6.5957 - accuracy: 0.660 - ETA: 18s - loss: 6.5933 - accuracy: 0.660 - ETA: 17s - loss: 6.5907 - accuracy: 0.660 - ETA: 17s - loss: 6.5903 - accuracy: 0.660 - ETA: 17s - loss: 6.5864 - accuracy: 0.660 - ETA: 17s - loss: 6.5846 - accuracy: 0.660 - ETA: 16s - loss: 6.5818 - accuracy: 0.660 - ETA: 16s - loss: 6.5793 - accuracy: 0.660 - ETA: 16s - loss: 6.5772 - accuracy: 0.660 - ETA: 15s - loss: 6.5754 - accuracy: 0.661 - ETA: 15s - loss: 6.5733 - accuracy: 0.661 - ETA: 15s - loss: 6.5731 - accuracy: 0.661 - ETA: 15s - loss: 6.5727 - accuracy: 0.661 - ETA: 14s - loss: 6.5703 - accuracy: 0.661 - ETA: 14s - loss: 6.5691 - accuracy: 0.661 - ETA: 14s - loss: 6.5684 - accuracy: 0.661 - ETA: 14s - loss: 6.5682 - accuracy: 0.661 - ETA: 13s - loss: 6.5654 - accuracy: 0.661 - ETA: 13s - loss: 6.5639 - accuracy: 0.661 - ETA: 13s - loss: 6.5653 - acc

...

20000/20000 [==============================] - ETA: 18s - loss: 4.8256 - accuracy: 0.695 - ETA: 17s - loss: 4.8244 - accuracy: 0.694 - ETA: 17s - loss: 4.8260 - accuracy: 0.695 - ETA: 17s - loss: 4.8259 - accuracy: 0.694 - ETA: 17s - loss: 4.8263 - accuracy: 0.695 - ETA: 16s - loss: 4.8267 - accuracy: 0.694 - ETA: 16s - loss: 4.8259 - accuracy: 0.694 - ETA: 16s - loss: 4.8250 - accuracy: 0.694 - ETA: 15s - loss: 4.8252 - accuracy: 0.694 - ETA: 15s - loss: 4.8253 - accuracy: 0.694 - ETA: 15s - loss: 4.8247 - accuracy: 0.694 - ETA: 15s - loss: 4.8245 - accuracy: 0.694 - ETA: 14s - loss: 4.8249 - accuracy: 0.694 - ETA: 14s - loss: 4.8231 - accuracy: 0.694 - ETA: 14s - loss: 4.8218 - accuracy: 0.694 - ETA: 13s - loss: 4.8203 - accuracy: 0.694 - ETA: 13s - loss: 4.8187 - accuracy: 0.694 - ETA: 13s - loss: 4.8175 - accuracy: 0.693 - ETA: 13s - loss: 4.8160 - accuracy: 0.693 - ETA: 12s - loss: 4.8145 - accuracy: 0.694 - ETA: 12s - loss: 4.8135 - accuracy: 0.694 - ETA: 12s - loss: 4.8130 - acc

 5952/20000 [=======>......................] - ETA: 2:51 - loss: 5.2402 - accuracy: 0.71 - ETA: 2:49 - loss: 4.7774 - accuracy: 0.60 - ETA: 2:48 - loss: 4.9516 - accuracy: 0.61 - ETA: 2:47 - loss: 4.7833 - accuracy: 0.63 - ETA: 2:47 - loss: 4.7163 - accuracy: 0.64 - ETA: 2:47 - loss: 4.6939 - accuracy: 0.64 - ETA: 2:46 - loss: 4.6074 - accuracy: 0.65 - ETA: 2:46 - loss: 4.6293 - accuracy: 0.67 - ETA: 2:46 - loss: 4.5827 - accuracy: 0.65 - ETA: 2:47 - loss: 4.6599 - accuracy: 0.66 - ETA: 2:46 - loss: 4.6289 - accuracy: 0.67 - ETA: 2:46 - loss: 4.6616 - accuracy: 0.67 - ETA: 2:45 - loss: 4.6383 - accuracy: 0.67 - ETA: 2:45 - loss: 4.5712 - accuracy: 0.68 - ETA: 2:45 - loss: 4.5658 - accuracy: 0.68 - ETA: 2:45 - loss: 4.6299 - accuracy: 0.69 - ETA: 2:44 - loss: 4.6951 - accuracy: 0.68 - ETA: 2:44 - loss: 4.6732 - accuracy: 0.69 - ETA: 2:44 - loss: 4.6358 - accuracy: 0.68 - ETA: 2:43 - loss: 4.7020 - accuracy: 0.68 - ETA: 2:43 - loss: 4.7397 - accuracy: 0.69 - ETA: 2:43 - loss: 4.7377 - ac

11904/20000 [================>.............] - ETA: 1:59 - loss: 4.7558 - accuracy: 0.68 - ETA: 1:58 - loss: 4.7573 - accuracy: 0.68 - ETA: 1:58 - loss: 4.7554 - accuracy: 0.68 - ETA: 1:58 - loss: 4.7547 - accuracy: 0.68 - ETA: 1:57 - loss: 4.7569 - accuracy: 0.68 - ETA: 1:57 - loss: 4.7588 - accuracy: 0.68 - ETA: 1:57 - loss: 4.7546 - accuracy: 0.68 - ETA: 1:57 - loss: 4.7561 - accuracy: 0.68 - ETA: 1:56 - loss: 4.7587 - accuracy: 0.68 - ETA: 1:56 - loss: 4.7548 - accuracy: 0.68 - ETA: 1:56 - loss: 4.7555 - accuracy: 0.68 - ETA: 1:56 - loss: 4.7617 - accuracy: 0.68 - ETA: 1:55 - loss: 4.7668 - accuracy: 0.68 - ETA: 1:55 - loss: 4.7609 - accuracy: 0.68 - ETA: 1:55 - loss: 4.7617 - accuracy: 0.68 - ETA: 1:54 - loss: 4.7612 - accuracy: 0.68 - ETA: 1:54 - loss: 4.7600 - accuracy: 0.68 - ETA: 1:54 - loss: 4.7606 - accuracy: 0.68 - ETA: 1:54 - loss: 4.7579 - accuracy: 0.68 - ETA: 1:53 - loss: 4.7571 - accuracy: 0.68 - ETA: 1:53 - loss: 4.7531 - accuracy: 0.68 - ETA: 1:53 - loss: 4.7489 - ac

17856/20000 [=========================>....] - ETA: 1:08 - loss: 4.7316 - accuracy: 0.69 - ETA: 1:08 - loss: 4.7350 - accuracy: 0.69 - ETA: 1:07 - loss: 4.7387 - accuracy: 0.68 - ETA: 1:07 - loss: 4.7350 - accuracy: 0.69 - ETA: 1:07 - loss: 4.7333 - accuracy: 0.69 - ETA: 1:07 - loss: 4.7317 - accuracy: 0.69 - ETA: 1:06 - loss: 4.7296 - accuracy: 0.69 - ETA: 1:06 - loss: 4.7280 - accuracy: 0.69 - ETA: 1:06 - loss: 4.7308 - accuracy: 0.68 - ETA: 1:06 - loss: 4.7308 - accuracy: 0.69 - ETA: 1:05 - loss: 4.7321 - accuracy: 0.69 - ETA: 1:05 - loss: 4.7319 - accuracy: 0.69 - ETA: 1:05 - loss: 4.7303 - accuracy: 0.69 - ETA: 1:04 - loss: 4.7327 - accuracy: 0.69 - ETA: 1:04 - loss: 4.7327 - accuracy: 0.69 - ETA: 1:04 - loss: 4.7346 - accuracy: 0.68 - ETA: 1:04 - loss: 4.7338 - accuracy: 0.68 - ETA: 1:03 - loss: 4.7343 - accuracy: 0.68 - ETA: 1:03 - loss: 4.7331 - accuracy: 0.68 - ETA: 1:03 - loss: 4.7320 - accuracy: 0.68 - ETA: 1:03 - loss: 4.7304 - accuracy: 0.68 - ETA: 1:02 - loss: 4.7297 - ac

20000/20000 [==============================] - ETA: 17s - loss: 4.7156 - accuracy: 0.693 - ETA: 17s - loss: 4.7145 - accuracy: 0.693 - ETA: 17s - loss: 4.7168 - accuracy: 0.693 - ETA: 17s - loss: 4.7164 - accuracy: 0.693 - ETA: 16s - loss: 4.7152 - accuracy: 0.693 - ETA: 16s - loss: 4.7161 - accuracy: 0.694 - ETA: 16s - loss: 4.7172 - accuracy: 0.694 - ETA: 16s - loss: 4.7155 - accuracy: 0.694 - ETA: 15s - loss: 4.7166 - accuracy: 0.694 - ETA: 15s - loss: 4.7151 - accuracy: 0.694 - ETA: 15s - loss: 4.7167 - accuracy: 0.694 - ETA: 14s - loss: 4.7155 - accuracy: 0.694 - ETA: 14s - loss: 4.7144 - accuracy: 0.694 - ETA: 14s - loss: 4.7140 - accuracy: 0.693 - ETA: 14s - loss: 4.7140 - accuracy: 0.694 - ETA: 13s - loss: 4.7131 - accuracy: 0.694 - ETA: 13s - loss: 4.7139 - accuracy: 0.694 - ETA: 13s - loss: 4.7148 - accuracy: 0.694 - ETA: 13s - loss: 4.7143 - accuracy: 0.693 - ETA: 12s - loss: 4.7147 - accuracy: 0.693 - ETA: 12s - loss: 4.7149 - accuracy: 0.693 - ETA: 12s - loss: 4.7145 - acc

In [79]:
score = model.evaluate(x_dev_, y_dev_, batch_size=32)
score

4106/4106 [==============================] - ETA: 29 - ETA: 20 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ET

[4.524983886852533, 0.6875304579734802]

In [80]:
y_test = model.predict(x_test, batch_size=32)

In [103]:
y_test[4] > 0.5

array([ True, False, False, False,  True, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

## MLKNN Model

In [22]:
classifier = MLkNN(k=20)

In [23]:
x_train_ = x_train.reshape(24106, 60000)
x_train_.shape
x_train_[0].shape

(24106, 60000)

(60000,)

In [25]:
classifier.fit(x_train_[:10000], y_train[:10000])

MemoryError: Unable to allocate array with shape (10000, 60000) and data type float64

In [27]:
predictions = classifier.predict(x_test)

accuracy_score(y_test,predictions)

ValueError: Found array with dim 3. Estimator expected <= 2.